<head>
  <meta name="author" content="Rogério de Oliveira">
  <meta institution="author" content="ITM">
</head>

<img src="https://maua.br/images/selo-60-anos-maua.svg" width=300, align="right">
<!-- <h1 align=left><font size = 6, style="color:rgb(200,0,0)"> optional title </font></h1> -->


# **Lab: Hot Encode, Normalização, Regressão Logística e K-Vizinhos mais Próximos**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
#@markdown Prepare Datasets - **Just Run Before Start**
df = sns.load_dataset('tips')
from sklearn.model_selection import train_test_split

df, df_test = train_test_split(df, test_size=3, random_state=42)

df = df.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

df.to_csv('tips_train.csv', index=False)
df_test.to_csv('tips_test.csv', index=False)

df = sns.load_dataset('tips')
from sklearn.model_selection import train_test_split

df['tip_label'] = np.where(df['tip'] / df['total_bill'] > 0.12, '> 0.12', '<=0.12')
df['credit_card'] = np.random.randint(0, 10000, df.shape[0])
df['Sunday'] = df['day'] == 'Sun'
df['Sunday'] = df['Sunday'].astype(int)

df.drop(['tip', 'smoker', 'day'], axis=1, inplace=True)
df = df[['credit_card', 'total_bill', 'Sunday', 'sex', 'time', 'size', 'tip_label']]

df, df_test = train_test_split(df, test_size=3, random_state=42)
df = df.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

df['sex'] = (df['sex'] == 'Female').astype('int')
df['time'] = (df['time'] == 'Dinner').astype('int')
df_test['sex'] = (df_test['sex'] == 'Female').astype('int')
df_test['time'] = (df_test['time'] == 'Dinner').astype('int')

df.to_csv('tips_train_2.csv', index=False)
df_test.to_csv('tips_test_2.csv', index=False)

In [ ]:
df = pd.read_csv('tips_train.csv')
df_test = pd.read_csv('tips_test.csv')
display(df.head())
display(df_test.head())

,total_bill,tip,sex,smoker,day,time,size
0,25.89,5.16,Male,Yes,Sat,Dinner,4
1,13.00,2.00,Female,Yes,Thur,Lunch,2
2,17.89,2.00,Male,Yes,Sun,Dinner,2
3,28.44,2.56,Male,Yes,Thur,Lunch,2
4,12.48,2.52,Female,No,Thur,Lunch,2


,total_bill,tip,sex,smoker,day,time,size
0,19.82,3.18,Male,No,Sat,Dinner,2
1,8.77,2.00,Male,No,Sun,Dinner,2
2,24.55,2.00,Male,No,Sun,Dinner,4


# 1. Encode

Transforme `df` (`tips_train.csv`) aplicando o hot encode dos atributos não numéricos. Com o mesmo estimador gerado, faça a mesma transformação em `df_test` (`tips_test.csv`).

**Q1.** Qual o número final de atributos em `df`?

**Q2.** Qual a soma de todos os valores de `df_test` transformado? (`df_test.sum().sum()`).


In [ ]:
print(df.isnull().sum())


total_bill    0
tip           0
sex           0
smoker        0
day           0
time          0
size          0
dtype: int64


In [ ]:
print(df_test.isnull().sum())


total_bill    0
tip           0
sex           0
smoker        0
day           0
time          0
size          0
dtype: int64


In [ ]:
df.shape

(241, 7)

In [ ]:
df_test.shape

(3, 7)

In [ ]:
df_test.head()

,total_bill,tip,sex,smoker,day,time,size
0,19.82,3.18,Male,No,Sat,Dinner,2
1,8.77,2.00,Male,No,Sun,Dinner,2
2,24.55,2.00,Male,No,Sun,Dinner,4


In [ ]:
from sklearn.preprocessing import OneHotEncoder

# Cria o OneHotEncoder
encoder = OneHotEncoder(handle_unknown='ignore', drop='first', sparse_output=False)

# Aplica o OneHotEncoder nas colunas categóricas
encoder.fit(df[['smoker', 'sex', 'day',"time"]]) ## CUIDADO NÃO FAZER 2 FIT --> tira duas categorias no lugar de uma
encoded_data = encoder.transform(df[['smoker', 'sex', 'day',"time"]])

encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out())

# Concatena o DataFrame original (sem as colunas categóricas) com o DataFrame codificado
df_encoded = pd.concat([df.drop(['smoker', 'sex', 'day',"time"], axis=1), encoded_df], axis=1)

df_encoded.head()

,total_bill,tip,size,smoker_Yes,sex_Male,day_Sat,day_Sun,day_Thur,time_Lunch
0,25.89,5.16,4,1.0,1.0,1.0,0.0,0.0,0.0
1,13.00,2.00,2,1.0,0.0,0.0,0.0,1.0,1.0
2,17.89,2.00,2,1.0,1.0,0.0,1.0,0.0,0.0
3,28.44,2.56,2,1.0,1.0,0.0,0.0,1.0,1.0
4,12.48,2.52,2,0.0,0.0,0.0,0.0,1.0,1.0


In [ ]:
df_encoded.shape

(241, 9)

In [ ]:
encoded_data_teste = encoder.transform(df_test[['smoker', 'sex', 'day',"time"]])

encoded_df_test = pd.DataFrame(encoded_data_teste, columns=encoder.get_feature_names_out())

# Concatena o DataFrame original (sem as colunas categóricas) com o DataFrame codificado
df_test_encoded = pd.concat([df_test.drop(['smoker', 'sex', 'day',"time"], axis=1), encoded_df_test], axis=1)

df_test_encoded.head()

,total_bill,tip,size,smoker_Yes,sex_Male,day_Sat,day_Sun,day_Thur,time_Lunch
0,19.82,3.18,2,0.0,1.0,1.0,0.0,0.0,0.0
1,8.77,2.00,2,0.0,1.0,0.0,1.0,0.0,0.0
2,24.55,2.00,4,0.0,1.0,0.0,1.0,0.0,0.0


In [ ]:
df_test_encoded.sum().sum()

74.32

In [ ]:
# df = pd.read_csv('tips_train.csv')
# df_test = pd.read_csv('tips_test.csv')
# display(df.head())
# display(df_test.head())

In [ ]:
# pd.get_dummies(df_test, columns=['smoker', 'sex', 'day',"time"], drop_first=True).sum().sum()

In [ ]:
# pd.get_dummies(df_test, columns=['smoker', 'sex', 'day',"time"], drop_first=True)

In [ ]:
df_test

,total_bill,tip,sex,smoker,day,time,size
0,19.82,3.18,Male,No,Sat,Dinner,2
1,8.77,2.00,Male,No,Sun,Dinner,2
2,24.55,2.00,Male,No,Sun,Dinner,4


In [ ]:
df_test.drop(columns=['smoker', 'sex', 'day',"time"]).sum().sum()

68.32

In [ ]:
68.32+4*3

80.32

In [ ]:
encoded_df_test.sum().sum()+df_test.drop(columns=['smoker', 'sex', 'day',"time"]).sum().sum()

74.32

# 2. Normalização

Normalize todos os atributos numérico de `df` (`tips_train.csv`) aplicando a normalização *z-score*. Com o mesmo estimador normalize os valores de `df_test` (`tips_test.csv`).

**Q3.** Qual o menor valor de `total_bill` no dataframe `df_test` (`tips_test.csv`) transformado?

<br>

**Nota**: Este exercício é independente do anterior. Carregue novamente os dados, não empregue os dados transformados.

conferindo

In [ ]:
df = pd.read_csv('tips_train.csv')
df_test = pd.read_csv('tips_test.csv')
display(df.head())
display(df_test.head())

,total_bill,tip,sex,smoker,day,time,size
0,25.89,5.16,Male,Yes,Sat,Dinner,4
1,13.00,2.00,Female,Yes,Thur,Lunch,2
2,17.89,2.00,Male,Yes,Sun,Dinner,2
3,28.44,2.56,Male,Yes,Thur,Lunch,2
4,12.48,2.52,Female,No,Thur,Lunch,2


,total_bill,tip,sex,smoker,day,time,size
0,19.82,3.18,Male,No,Sat,Dinner,2
1,8.77,2.00,Male,No,Sun,Dinner,2
2,24.55,2.00,Male,No,Sun,Dinner,4


In [ ]:
df[['total_bill', 'tip']]

In [ ]:
df["normBill"] = df['total_bill']-df['total_bill'].mean()/df['total_bill'].std()
df["normTip"] = df['tip']-df['tip'].mean()/df['tip'].std()

In [ ]:
df


,total_bill,tip,sex,smoker,day,time,size,normBill,normTip
0,25.89,5.16,Male,Yes,Sat,Dinner,4,23.670017,2.996307
1,13.00,2.00,Female,Yes,Thur,Lunch,2,10.780017,-0.163693
2,17.89,2.00,Male,Yes,Sun,Dinner,2,15.670017,-0.163693
3,28.44,2.56,Male,Yes,Thur,Lunch,2,26.220017,0.396307
4,12.48,2.52,Female,No,Thur,Lunch,2,10.260017,0.356307
...,...,...,...,...,...,...,...,...,...
236,20.49,4.06,Male,Yes,Sat,Dinner,2,18.270017,1.896307
237,14.83,3.02,Female,No,Sun,Dinner,2,12.610017,0.856307
238,5.75,1.00,Female,Yes,Fri,Dinner,2,3.530017,-1.163693
239,34.63,3.55,Male,Yes,Sun,Dinner,2,32.410017,1.386307


In [ ]:
df = pd.read_csv('tips_train.csv')
df_test = pd.read_csv('tips_test.csv')
display(df.head())
display(df_test.head())

,total_bill,tip,sex,smoker,day,time,size
0,25.89,5.16,Male,Yes,Sat,Dinner,4
1,13.00,2.00,Female,Yes,Thur,Lunch,2
2,17.89,2.00,Male,Yes,Sun,Dinner,2
3,28.44,2.56,Male,Yes,Thur,Lunch,2
4,12.48,2.52,Female,No,Thur,Lunch,2


,total_bill,tip,sex,smoker,day,time,size
0,19.82,3.18,Male,No,Sat,Dinner,2
1,8.77,2.00,Male,No,Sun,Dinner,2
2,24.55,2.00,Male,No,Sun,Dinner,4


In [ ]:
df_test["normBill"] = df_test['total_bill']-df['total_bill'].mean()/df['total_bill'].std()
df_test["normTip"] = df_test['tip']-df['tip'].mean()/df['tip'].std()

In [ ]:
df_test

,total_bill,tip,sex,smoker,day,time,size,normBill,normTip
0,19.82,3.18,Male,No,Sat,Dinner,2,17.600017,1.016307
1,8.77,2.00,Male,No,Sun,Dinner,2,6.550017,-0.163693
2,24.55,2.00,Male,No,Sun,Dinner,4,22.330017,-0.163693


In [ ]:
df_test.normBill.min()

6.5500171984641185

In [ ]:
df = pd.read_csv('tips_train.csv')
df_test = pd.read_csv('tips_test.csv')
display(df.head())
display(df_test.head())

,total_bill,tip,sex,smoker,day,time,size
0,25.89,5.16,Male,Yes,Sat,Dinner,4
1,13.00,2.00,Female,Yes,Thur,Lunch,2
2,17.89,2.00,Male,Yes,Sun,Dinner,2
3,28.44,2.56,Male,Yes,Thur,Lunch,2
4,12.48,2.52,Female,No,Thur,Lunch,2


,total_bill,tip,sex,smoker,day,time,size
0,19.82,3.18,Male,No,Sat,Dinner,2
1,8.77,2.00,Male,No,Sun,Dinner,2
2,24.55,2.00,Male,No,Sun,Dinner,4


In [ ]:
df_test["normBill"] = df_test['total_bill']-df_test['total_bill'].mean()/df_test['total_bill'].std()
df_test["normTip"] = df_test['tip']-df_test['tip'].mean()/df_test['tip'].std()


In [ ]:
df_test

,total_bill,tip,sex,smoker,day,time,size,normBill,normTip
0,19.82,3.18,Male,No,Sat,Dinner,2,17.632679,-0.33303
1,8.77,2.00,Male,No,Sun,Dinner,2,6.582679,-1.51303
2,24.55,2.00,Male,No,Sun,Dinner,4,22.362679,-1.51303


fim conferindo

In [ ]:
df = pd.read_csv('tips_train.csv')
df_test = pd.read_csv('tips_test.csv')
display(df.head())
display(df_test.head())

,total_bill,tip,sex,smoker,day,time,size
0,25.89,5.16,Male,Yes,Sat,Dinner,4
1,13.00,2.00,Female,Yes,Thur,Lunch,2
2,17.89,2.00,Male,Yes,Sun,Dinner,2
3,28.44,2.56,Male,Yes,Thur,Lunch,2
4,12.48,2.52,Female,No,Thur,Lunch,2


,total_bill,tip,sex,smoker,day,time,size
0,19.82,3.18,Male,No,Sat,Dinner,2
1,8.77,2.00,Male,No,Sun,Dinner,2
2,24.55,2.00,Male,No,Sun,Dinner,4


In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler

standard_scaler = StandardScaler()


df_scaled = pd.DataFrame(standard_scaler.fit_transform(df[['total_bill', 'tip']]),
                        columns=['total_bill', 'tip'])


df_scaled = pd.concat([df[['smoker', 'sex', 'day',"time","size"]], df_scaled], axis=1)
df_scaled.head()

,smoker,sex,day,time,size,total_bill,tip
0,Yes,Male,Sat,Dinner,4,0.682510,1.553893
1,Yes,Female,Thur,Lunch,2,-0.764871,-0.725525
2,Yes,Male,Sun,Dinner,2,-0.215787,-0.725525
3,Yes,Male,Thur,Lunch,2,0.968842,-0.321578
4,No,Female,Thur,Lunch,2,-0.823260,-0.350431


In [ ]:
df_test_scaled = pd.DataFrame(standard_scaler.transform(df_test[['total_bill', 'tip']]),
                        columns=['total_bill', 'tip'])



df_test_scaled = pd.concat([df_test[['smoker', 'sex', 'day',"time","size"]], df_test_scaled], axis=1)
df_test_scaled.head()

,smoker,sex,day,time,size,total_bill,tip
0,No,Male,Sat,Dinner,2,0.000927,0.125650
1,No,Male,Sun,Dinner,2,-1.239845,-0.725525
2,No,Male,Sun,Dinner,4,0.532045,-0.725525


In [ ]:
df_test_scaled.total_bill.min()

-1.2398451707329228

# 3. Regressão Logística

Faça um modelo de **regressão Logística** para prever os valores de `tip_label` **com base nos demais atributos que você considere preditores** (tenha atenção aqui, nem todos os atributos são preditores). Para a separação de dados de treinamento e teste, empregue:
```
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
```
e empregue o **número máximo de iterações do estimador logístico em 1000**.

**Q4.** Qual a acuracidade do modelo obtida?

**Q5.** Qual a predição obtida para os 3 novos casos?

<br>

**nota**: O encode dos dados já foi aqui realizado antes e não é necessário normalizar os dados para este exercício (a normalização não altera significativamente os resultados aqui).

In [ ]:
df = pd.read_csv('tips_train_2.csv')
df_new = pd.read_csv('tips_test_2.csv')
display(df.head())
display(df_new.head())

,credit_card,total_bill,Sunday,sex,time,size,tip_label
0,1200,25.89,0,0,1,4,> 0.12
1,9274,13.00,0,1,0,2,> 0.12
2,7343,17.89,1,0,1,2,<=0.12
3,4849,28.44,0,0,0,2,<=0.12
4,7055,12.48,0,1,0,2,> 0.12


,credit_card,total_bill,Sunday,sex,time,size,tip_label
0,3741,19.82,0,0,1,2,> 0.12
1,6744,8.77,1,0,1,2,> 0.12
2,7079,24.55,1,0,1,4,<=0.12


In [ ]:

# Definir entradas e saidas do modelo

X = df[['total_bill','Sunday',"sex","time","size"]]
y = df.tip_label

In [ ]:
X.head()

,total_bill,Sunday,sex,time,size
0,25.89,0,0,1,4
1,13.00,0,1,0,2
2,17.89,1,0,1,2
3,28.44,0,0,0,2
4,12.48,0,1,0,2


In [ ]:
y.head()

,tip_label
0,> 0.12
1,> 0.12
2,<=0.12
3,<=0.12
4,> 0.12


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(max_iter=1000)
print(clf)


LogisticRegression(max_iter=1000)


In [ ]:

# Treinar modelo
clf.fit(X_train,y_train)
print( clf.coef_, clf.intercept_)

[[-0.14540699 -0.01198749  0.42253708 -0.46588612  0.59058569]] [3.21990905]


In [ ]:

# Acuracidade
accuracy = clf.score(X_test, y_test)
accuracy

0.8571428571428571

In [ ]:
df_new

,credit_card,total_bill,Sunday,sex,time,size,tip_label
0,3741,19.82,0,0,1,2,> 0.12
1,6744,8.77,1,0,1,2,> 0.12
2,7079,24.55,1,0,1,4,<=0.12


In [ ]:

# Prever baseado no modelo

y_pred = clf.predict(df_new[['total_bill','Sunday',"sex","time","size"]])
print(y_pred)


['> 0.12' '> 0.12' '> 0.12']


# 4. K-Vizinhos mais Próximos

Faça um modelo de **K-Vizinhos mais Próximos** para prever os valores de `tip_label` **com base nos demais atributos que você considere preditores** (tenha atenção aqui, nem todos os atributos são preditores). Para a separação de dados de treinamento e teste, empregue:
```
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
```
e empregue **k=9**.

**Q6.** Qual a acuracidade do modelo obtida?

**Q7.** Qual a predição obtida para os 3 novos casos?

**Q8.** Verifique qual classe apresenta mais **falsos negativos**. Qual é esse número de falsos negativos?

<br>

**nota**: O encode dos dados já foi aqui realizado antes e não é necessário normalizar os dados para este exercício (a normalização não altera significativamente os resultados aqui).

In [ ]:
df = pd.read_csv('tips_train_2.csv')
df_new = pd.read_csv('tips_test_2.csv')
display(df.head())
display(df_new.head())

,credit_card,total_bill,Sunday,sex,time,size,tip_label
0,1200,25.89,0,0,1,4,> 0.12
1,9274,13.00,0,1,0,2,> 0.12
2,7343,17.89,1,0,1,2,<=0.12
3,4849,28.44,0,0,0,2,<=0.12
4,7055,12.48,0,1,0,2,> 0.12


,credit_card,total_bill,Sunday,sex,time,size,tip_label
0,3741,19.82,0,0,1,2,> 0.12
1,6744,8.77,1,0,1,2,> 0.12
2,7079,24.55,1,0,1,4,<=0.12


In [ ]:
from sklearn import neighbors
from sklearn.preprocessing import MinMaxScaler

In [ ]:

# Definir entradas e saidas do modelo

X = df[['total_bill','Sunday',"sex","time","size"]]
y = df.tip_label

In [ ]:
X.head()

,total_bill,Sunday,sex,time,size
0,25.89,0,0,1,4
1,13.00,0,1,0,2
2,17.89,1,0,1,2
3,28.44,0,0,0,2
4,12.48,0,1,0,2


In [ ]:
y.head()

,tip_label
0,> 0.12
1,> 0.12
2,<=0.12
3,<=0.12
4,> 0.12


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:



clf = neighbors.KNeighborsClassifier(n_neighbors = 9)

clf.fit(X_train, y_train)


KNeighborsClassifier(n_neighbors=9)

In [ ]:

# Acuracidade
accuracy = clf.score(X_test, y_test)
accuracy

0.8163265306122449

In [ ]:

y_pred = clf.predict(df_new[['total_bill','Sunday',"sex","time","size"]])

default_pred = y_pred
print(y_pred)

['> 0.12' '> 0.12' '> 0.12']


In [ ]:
y_pred = clf.predict(X_test[['total_bill','Sunday',"sex","time","size"]])


In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

cm = confusion_matrix(y_test, y_pred)
print('\nMatriz de Confusão:\n')
print(cm)

# accuracy = accuracy_score(y_test, y_pred)
# print('\nScore de Acuracidade (1):\n')
# print(f'{accuracy:.2f}')

# accuracy = clf.score(X_test, y_test)
# print('\nScore de Acuracidade (2):\n')
# print(f'{accuracy:.2f}')

# print('\nClassification Report:\n')
# print(classification_report(y_test, y_pred))


Matriz de Confusão:

[[ 1  6]
 [ 3 39]]


In [ ]:
import pandas as pd

# Criando os dados da matriz de confusão com as siglas
data = [
    ['TP', 'FN'],
    ['FP', 'TN']
]

# Criando um DataFrame com dois níveis de cabeçalhos
df_confusion = pd.DataFrame(data,
                            columns=pd.MultiIndex.from_tuples(
                                [('Valor Previsto', 'Positivo'), ('Valor Previsto', 'Negativo')]
                            ),
                            index=pd.MultiIndex.from_tuples(
                                [('Valor Real', 'Positivo'), ('Valor Real', 'Negativo')]
                            ))

# df_confusion

# Função para aplicar cores conforme o tipo de previsão
def color_confusion(val):
    if val == 'TP':
        color = 'green'
    elif val == 'FP':
        color = 'red'
    elif val == 'FN':
        color = 'gray'
    elif val == 'TN':
        color = 'blue'
    else:
        color = 'white'
    return f'background-color: {color}'
# Aplicando o estilo com as cores
styled_df = df_confusion.style.applymap(color_confusion)

# Exibindo o DataFrame estilizado
styled_df


<ipython-input-66-fd5c976051a5>:34: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  styled_df = df_confusion.style.applymap(color_confusion)


In [ ]:
18+9

27